1. NoSQL Databases:
   a. Write a Python program that connects to a MongoDB database and inserts a new document into a collection named "students". The document should include fields such as "name", "age", and "grade". Print a success message after the insertion.
   b. Implement a Python function that connects to a Cassandra database and inserts a new record into a table named "products". The record should contain fields like "id", "name", and "price". Handle any potential errors that may occur during the insertion.



In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['students']

document = {
    'name': 'John Doe',
    'age': 20,
    'grade': 'A'
}

collection.insert_one(document)

print("Document inserted successfully.")


from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel

def insert_product(id, name, price):
    try:
        cluster = Cluster(['127.0.0.1'])
        session = cluster.connect()
        session.set_keyspace('mykeyspace')

        insert_query = session.prepare("INSERT INTO products (id, name, price) VALUES (?, ?, ?)")
        insert_query.consistency_level = ConsistencyLevel.QUORUM

        session.execute(insert_query, (id, name, price))

        print("Record inserted successfully.")
    except Exception as e:
        print("Error occurred:", str(e))
    finally:
        session.shutdown()
        cluster.shutdown()


2. Document-oriented NoSQL Databases:
   a. Given a MongoDB collection named "books", write a Python function that fetches all the books published in the last year and prints their titles and authors.
   b. Design a schema for a document-oriented NoSQL database to store customer information for an e-commerce platform. Write a Python program to insert a new customer document into the database and handle any necessary validations.





In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta

def fetch_recent_books():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['mydatabase']
    collection = db['books']

    one_year_ago = datetime.now() - timedelta(days=365)

    recent_books = collection.find({"publication_date": {"$gte": one_year_ago}})

    for book in recent_books:
        print("Title:", book['title'])
        print("Author:", book['author'])
        print()

    client.close()

from pymongo import MongoClient

client = MongoClient('mongodb://primary_node:27017,secondary_node1:27017,secondary_node2:27017/?replicaSet=myreplicaset')

status = client.admin.command('replSetGetStatus')

for member in status['members']:
    if member['stateStr'] == 'PRIMARY':
        print("Primary Node:", member['name'])
    elif member['stateStr'] == 'SECONDARY':
        print("Secondary Node:", member['name'])

client.close()

from cassandra.cluster import Cluster

cluster = Cluster(['node1', 'node2', 'node3'])
session = cluster.connect()

rows = session.execute("SELECT * FROM system.peers")

for row in rows:
    print("Node:", row.peer, "Status:", row.status)

session.shutdown()
cluster.shutdown()


3. High Availability and Fault Tolerance:
   a. Explain the concept of replica sets in MongoDB. Write a Python program that connects to a MongoDB replica set and retrieves the status of the primary and secondary nodes.
   b. Describe how Cassandra ensures high availability and fault tolerance in a distributed database system. Write a Python program that connects to a Cassandra cluster and fetches the status of the nodes.



In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId

client = MongoClient('mongodb://mongos1:27017')

admin_db = client.admin
admin_db.command("enableSharding", "mydatabase")

shard_key = {'_id': 'hashed'}
admin_db.command("shardCollection", "mydatabase.students", key=shard_key)

db = client.mydatabase
collection = db.students

documents = [
    {'_id': ObjectId(), 'name': 'John Doe', 'age': 20, 'grade': 'A'},
    {'_id': ObjectId(), 'name': 'Jane Smith', 'age': 18, 'grade': 'B'}
]

collection.insert_many(documents)

client.close()

from pymongo import MongoClient
from bson.objectid import ObjectId

client = MongoClient('mongodb://mongos1:27017')

admin_db = client.admin
admin_db.command("enableSharding", "mydatabase")

shard_key = {'user_id': 1}
admin_db.command("shardCollection", "mydatabase.users", key=shard_key)

db = client.mydatabase
collection = db.users

users = [
    {'_id': ObjectId(), 'user_id': 1001, 'name': 'John Doe', 'email': 'john@example.com'},
    {'_id': ObjectId(), 'user_id': 2002, 'name': 'Jane Smith', 'email': 'jane@example.com'}
]

collection.insert_many(users)

sharded_query = {'user_id': 1001}
user_data = collection.find_one(sharded_query)
print(user_data)

client.close()


4. Sharding in MongoDB:
   a. Explain the concept of sharding in MongoDB and how it improves performance and scalability. Write a Python program that sets up sharding for a MongoDB cluster and inserts multiple documents into a sharded collection.
   b. Design a sharding strategy for a social media application where user data needs to be distributed across multiple shards. Write a Python program to demonstrate how data is distributed and retrieved from the sharded cluster.



In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['products']

collection.create_index('name')

query = {'name': 'example_product'}
result = collection.find(query)

for doc in result:
    print(doc)

client.close()



5. Indexing in MongoDB:
   a. Describe the concept of indexing in MongoDB and its importance in query optimization. Write a Python program that creates an index on a specific field in a MongoDB collection and executes a query using that index.
   b. Given a MongoDB collection named "products", write a Python function that searches for products with a specific keyword in the name or description. Optimize the query by adding appropriate indexes.

In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['products']

collection.create_index('name')
collection.create_index('description')

def search_products(keyword):
    query = {
        '$or': [
            {'name': {'$regex': keyword, '$options': 'i'}},
            {'description': {'$regex': keyword, '$options': 'i'}}
        ]
    }
    result = collection.find(query)

    for doc in result:
        print(doc)

client.close()
